# Echo TTS Streaming Server - Colab

Run Echo TTS server on Colab with Cloudflare tunnel for use with Pipecat voice agents.

## Step 1: Setup Environment


In [ ]:
# Check for GPU
import torch

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"✅ GPU detected: {gpu_name}")
else:
    print("⚠️ No GPU detected! Enable GPU in Runtime > Change runtime type > T4 GPU")


In [ ]:
# Clone Echo TTS repository
import os
import subprocess

# UPDATE THIS URL to your repo
REPO_URL = "https://github.com/YOUR_USERNAME/echo-tts-api.git"
REPO_DIR = "/content/echo-tts-api"

if not os.path.exists(REPO_DIR):
    subprocess.run(["git", "clone", "--quiet", "--depth", "1", REPO_URL, REPO_DIR], check=True)
    print(f"✅ Cloned Echo TTS repository")
else:
    print(f"✅ Repository already exists")
    subprocess.run(["git", "-C", REPO_DIR, "pull", "--quiet"], check=True)
    print("   Pulled latest changes")


In [ ]:
# Install dependencies
import subprocess
import sys
import urllib.request
import stat

# Install PyTorch with CUDA
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "torch", "torchaudio", "--index-url", "https://download.pytorch.org/whl/cu121"], check=True)
print("✅ Installed PyTorch")

# Install requirements
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "-r", f"{REPO_DIR}/requirements.txt"], check=True)
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "websockets"], check=True)
print("✅ Installed Python dependencies")

# Download cloudflared
urllib.request.urlretrieve(
    "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64",
    "/content/cloudflared"
)
os.chmod("/content/cloudflared", stat.S_IRWXU)
print("✅ Downloaded cloudflared")


In [ ]:
# Pre-download models (so you can see the progress)
from huggingface_hub import snapshot_download

print("📥 Downloading Echo TTS model (~2GB)...")
snapshot_download("jordand/echo-tts-base", local_dir_use_symlinks=False)
print("✅ Downloaded echo-tts-base")

print("📥 Downloading audio decoder...")
snapshot_download("jordand/fish-s1-dac-min", local_dir_use_symlinks=False)
print("✅ Downloaded fish-s1-dac-min")

print("\n✅ All models downloaded!")


## Step 2: Launch Echo TTS Server with Cloudflare Tunnel

This will start the server and create a public URL. First request may take 30-60s as model loads.


In [ ]:
# Launch server with Cloudflare tunnel
import subprocess
import re
import time
import threading
import os

os.chdir(REPO_DIR)

# Environment for Echo TTS
env = os.environ.copy()
env["ECHO_DEVICE"] = "cuda"
env["PORT"] = "8000"

# Start server
print("🚀 Starting Echo TTS server...")
srv = subprocess.Popen(
    f"python {REPO_DIR}/api_server.py",
    shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
    text=True, bufsize=1, env=env
)

# Start Cloudflare tunnel
print("🌐 Starting Cloudflare tunnel...")
cf = subprocess.Popen(
    "/content/cloudflared tunnel --url http://localhost:8000 --no-autoupdate",
    shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
    text=True, bufsize=1
)

public_url = None
url_pattern = re.compile(r"(https://[a-z0-9-]+\.trycloudflare\.com)")

def read_srv():
    for line in srv.stdout:
        if line.strip():
            print(f"[SERVER] {line.strip()}")

def read_cf():
    global public_url
    for line in cf.stdout:
        m = url_pattern.search(line)
        if m:
            public_url = m.group(1)
            break

threading.Thread(target=read_srv, daemon=True).start()
threading.Thread(target=read_cf, daemon=True).start()

print("\n⏳ Waiting for tunnel URL...\n")

displayed = False
while True:
    if public_url and not displayed:
        print("=" * 60)
        print(f"✅ PUBLIC URL: {public_url}")
        print("=" * 60)
        print(f"\n📋 For Pipecat, set:")
        print(f"   ECHO_SERVER_URL={public_url.replace('https://', 'wss://')}")
        print(f"\n⚠️  This URL is temporary!")
        print("=" * 60)
        displayed = True
        public_url = None
    
    if srv.poll() is not None:
        print("\n❌ Server exited")
        break
    if cf.poll() is not None:
        print("\n❌ Tunnel exited")
        break
    
    time.sleep(0.5)


## Step 3: Test Server (Optional)


In [ ]:
# Test the server
import requests

try:
    r = requests.get("http://localhost:8000/health")
    print(f"✅ Health: {r.json()}")
    
    r = requests.get("http://localhost:8000/v1/voices")
    voices = r.json().get("data", [])
    print(f"✅ Voices: {[v['id'] for v in voices[:5]]}...")
except Exception as e:
    print(f"❌ Error: {e}")
